In [ ]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


In [ ]:

books_df = pd.read_json('20s_best_book.json')
books = books_df[['no', 'ranking', 'bookname', 'authors', 'publisher', 'publication_year', 'isbn13']].copy()


In [ ]:

def get_page_and_weight(isbn):
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        r = requests.get(url.format(isbn), headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, 'html.parser')

        prd_info = soup.find('a', attrs={'class': 'gd_name'})
        if prd_info is None:
            return None, None

        detail_url = 'http://www.yes24.com' + prd_info['href']
        r = requests.get(detail_url, headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, 'html.parser')

        prd_detail = soup.find('div', attrs={'id': 'infoset_specific'})
        if prd_detail is None:
            return None, None

        prd_tr_list = prd_detail.find_all('tr')
        for tr in prd_tr_list:
            th = tr.find('th')
            td = tr.find('td')
            if th and th.get_text(strip=True) == '쪽수, 무게, 크기':
                info_parts = [part.strip() for part in td.get_text(strip=True).split('|')]
                page_cnt = int(''.join(filter(str.isdigit, info_parts[0]))) if len(info_parts) >= 1 else None
                weight_info = info_parts[1] if len(info_parts) >= 2 else None
                return page_cnt, weight_info
        return None, None
    except Exception as e:
        print(f"[예외 발생] ISBN {isbn}: {e}")
        return None, None


In [ ]:

page_list = []
weight_list = []


In [ ]:

for i, isbn in enumerate(books['isbn13']):
    print(f"[{i+1}] ISBN {isbn} 처리 중...")
    page, weight = get_page_and_weight(isbn)
    page_list.append(page)
    weight_list.append(weight)
    time.sleep(1)


In [ ]:

books['page_cnt'] = page_list
books['weight_info'] = weight_list


In [ ]:

books.head()
